In [34]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from scipy import io
from tensorflow.keras.callbacks import ModelCheckpoint

In [35]:
dataset = io.loadmat('datasets/emnist-letters.mat')
# matlab_structs / numpy structured arrays. yay
# they have a dtype (structure) and every element adheres to it :)

In [36]:
struct_dataset = dataset['dataset'][0,0]

train_images, train_labels = struct_dataset['train'][0,0]['images'], struct_dataset['train'][0,0]['labels']
test_images, test_labels = struct_dataset['test'][0,0]['images'], struct_dataset['test'][0,0]['labels']

print(train_images.shape, test_images.shape, train_labels.shape, test_labels.shape)

(124800, 784) (20800, 784) (124800, 1) (20800, 1)


In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), padding="same", use_bias="false", input_shape=(28,28,1)),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(32, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(32, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.35),
    layers.Dense(26, activation="softmax")
])

model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [38]:
train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = train_labels - 1
test_labels = test_labels - 1

train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

checkpoint = ModelCheckpoint(filepath="checkpoints/epoch_{epoch:02d}.keras", save_weights_only=False, save_freq="epoch")

model.fit(train_images, train_labels, epochs=15,
          validation_data=(test_images, test_labels), callbacks=[checkpoint], batch_size=64)

# Best = 94.620% validation acc

Epoch 1/15
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 149s 76ms/step - accuracy: 0.5007 - loss: 1.4704 - val_accuracy: 0.8557 - val_loss: 0.4942
Epoch 2/15
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 152s 78ms/step - accuracy: 0.6361 - loss: 1.0253 - val_accuracy: 0.8948 - val_loss: 0.3719
Epoch 3/15
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 162s 83ms/step - accuracy: 0.6821 - loss: 0.8915 - val_accuracy: 0.9118 - val_loss: 0.3044
Epoch 4/15
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 165s 85ms/step - accuracy: 0.7389 - loss: 0.7423 - val_accuracy: 0.9247 - val_loss: 0.2631
Epoch 5/15
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 168s 86ms/step - accuracy: 0.7635 - loss: 0.6761 - val_accuracy: 0.9303 - val_loss: 0.2334
Epoch 6/15
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 168s 86ms/step - accuracy: 0.8026 - loss: 0.5771 - val_accuracy: 0.9323 - val_loss: 0.2315
Epoch 7/15
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 166s 85ms/step - accuracy: 0.8406 - loss: 0.4737 - val_accuracy: 0.9374 - val_loss: 0.2136
Epoch 8/15
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 160s 82ms/step - accuracy: 